#Tree Visualisation
Extracting nodes and edges from the data and visualising it in Tree form in Latex.

In [ ]:
import networkx as nx

In [ ]:
# Step 1: Upload the CONLLU file
from IPython.display import display
from ipywidgets import FileUpload

# Create a file upload widget that accepts CONLLU files
upload = FileUpload(accept='.conllu', multiple=False)

# Display the upload widget
display(upload)


FileUpload(value={}, accept='.conllu', description='Upload')

In [ ]:
# Check if any files were uploaded
if upload.value:
    # Get the uploaded file content
    uploaded_file_content = upload.value[next(iter(upload.value))]['content']
    # Process the uploaded file content
    # (Your code to process the file goes here)
else:
    print("No file uploaded. Please upload a CoNLL-U format file.")


In [ ]:
!pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 2.2 MB/s eta 0:00:00


In [ ]:


class Compute_measures(object):
    def __init__(self, tree):
        self.tree=tree                          # tree encodes the nodes and edges content in dictionary format. It uses directed graph (DiGraph) feature of networkX package. For example, nodes are encoded like this - tree.nodes={1:{form:'this',POS:'PRN'},2:{...}}
        self.root=0                             # ROOT is an abstract node in the tree and is encoded as empty and with name=0


In [ ]:
# Packages used: NetworkX -- for tree representation as directed acyclic graphs

import networkx as nx

class Compute_measures(object):
    def __init__(self, tree):
        self.tree=tree                          # tree encodes the nodes and edges content in dictionary format. It uses directed graph (DiGraph) feature of networkX package. For example, nodes are encoded like this - tree.nodes={1:{form:'this',POS:'PRN'},2:{...}}
        self.root=0                             # ROOT is an abstract node in the tree and is encoded as empty and with name=0

    def dependency_direction(self, edge):       # Computes the direction of an edge (i.e., dependency) according to relative position of dependent and head
        direction=''
        if edge[0]>edge[1]:                     # edge is a list data type in the format [head,dependent]
            direction='RL'
        else:
            direction='LR'

        return direction                        # return direction as 'LR' (Left-to-Right) or RL ('Right-to-Left')

    def dependency_distance(self, edge):        # Computes the dependency length i.e., no. of nodes between head and its dependent
        dd=0
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:                             # all the nodes that lies linearly between dependent and head
                    dd+=1
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    dd+=1
        return dd                              # returns the dependency distance of the edge

    def dependency_depth(self, edge):        # Computes the dependency length i.e., no. of nodes between head and its dependent
        hd=1
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:                             # all the nodes that lies linearly between dependent and head
                    if nx.descendants(self.tree, nodex):
                        hd+=1
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    if nx.descendants(self.tree, nodex):
                        hd+=1
        return hd                              # returns the dependency distance of the edge


    def is_projective(self, edge):             # Checks if an edge is projective or not and returns a boolean value.
        projective=True
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        flag=0
        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:
                if not nodeI in nx.descendants(self.tree, edge[0]):
                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                        flag += 1
        if not flag==0:
            projective=False
        return projective

    def edge_degree(self, edge):                                 # Computes the number of edges causing non-projectivity
        eD=0
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:         # if the head of any intervening node exists outside the span of the edge
                if not nodeI in nx.descendants(self.tree, edge[0]):
                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                        eD += 1
        return eD

    def gap_degree(self, node):
        gapD = 0
        edge_span = []
        for nodex in nx.descendants(self.tree, self.root):
            if node == self.root or (self.tree.has_node(nodex) and 'head' in self.tree.nodes[nodex] and self.tree.nodes[nodex]['head'] is not None):
                if 'head' in self.tree.nodes[node] and self.tree.nodes[node]['head'] is not None:
                    if node < nodex < self.tree.nodes[node]['head']:
                        edge_span.append(nodex)
                    elif self.tree.nodes[node]['head'] < nodex < node:
                        edge_span.append(nodex)

        for nodeP in edge_span:
            if nodeP != self.root and 'head' in self.tree.nodes[nodeP] and self.tree.nodes[nodeP]['head'] is not None:
                if self.tree.nodes[nodeP]['head'] != self.root:
                    if not self.is_projective([self.tree.nodes[nodeP]['head'], nodeP]):
                        gapD += 1
        return gapD

    def gapnodes(self, pathx):                                          # Returns all the nodes which causes GAP in the projection of a node
        nodeM=[]
        for nodex in pathx:
            if not nodex==self.root:
                if not self.tree.nodes[nodex]['head']==self.root:
                    if not self.is_projective([self.tree.nodes[nodex]['head'],nodex]):
                        cross_dep=nodex
                        cross_head=self.tree.nodes[cross_dep]['head']
                        edge_span=[]
                        if cross_head>cross_dep:
                            for nodev in nx.descendants(self.tree, self.root):
                                if cross_dep<nodev<cross_head:
                                    edge_span.append(nodev)
                        else:
                            for nodev in nx.descendants(self.tree, self.root):
                                if cross_head<nodev<cross_dep:
                                    edge_span.append(nodev)
                        for nodeI in edge_span:
                            if not self.tree.nodes[nodeI]['head'] in edge_span:         # if the head of any intervening node exists outside the span of the edge
                                if not nodeI in nx.descendants(self.tree, cross_head):
                                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                                        nodeM.append(nodeI)
        return nodeM

    def illnestedness(self,node,gapD):
        k_illnest=0
        if gapD==0:
            k_illnest=0
        else:
            illnest = []
            all_gapped_chains=[]
            for nodex in nx.descendants(self.tree, self.root):
                if not nodex==self.root:
                    if not self.tree.nodes[nodex]['head']==self.root:
                        if not self.is_projective([self.tree.nodes[nodex]['head'],nodex]):
                            if nx.has_path(self.tree, node, nodex):
                                pathx=nx.all_simple_paths(self.tree, node, nodex, cutoff=None)
                                for item in pathx:
                                    all_gapped_chains.append(item)                             # It has all the projection chains from root to dependents of crossing arcs
                                                                                          # i.e., all possible chains which can interleave with other
            chains_with_gaps=[]
            for chainx in all_gapped_chains:
                flag=0
                for chainy in all_gapped_chains:
                    if set(chainx) < set(chainy):
                        flag=flag+1
                if flag==0:
                    chains_with_gaps.append(chainx)

            for pathz in chains_with_gaps:
                #print(pathz)
                num_interL=0                                                        # Variable for number of chains that can interleave with a single chain in question
                nodeM=self.gapnodes(pathz)
                for pathy in chains_with_gaps:
                    if not pathy==pathz:
                        flag=0
                        for nodeC in nodeM:
                            if nodeC in pathy:
                                flag=flag+1
                        if not flag==0:
                            nodeMM=self.gapnodes(pathy)
                            flagg=0
                            for nodeCC in nodeMM:
                                if nodeCC in pathz:
                                    flagg=flagg+1
                            if not flagg==0:
                                num_interL = num_interL + 1
                illnest.append(num_interL)
            k_illnest = max(illnest)
        return k_illnest

    def gapD_hist(self):
        gapd_histogram={}
        for nodex in self.tree.nodes:
            gapD=self.gap_degree(nodex)
            if gapD in gapd_histogram.keys():
                gapd_histogram[gapD]=gapd_histogram[gapD]+1
            else:
                gapd_histogram[gapD]=1
        return gapd_histogram

    def projection_degree(self, node):
        size_chains=[]
        terminals=[]
        for nodex in self.tree.nodes:
            if self.tree.out_degree(nodex)==0:
                terminals.append(nodex)

        for nodeT in terminals:
            size=0
            if nx.has_path(self.tree, node, nodeT):
                pathx=nx.all_simple_paths(self.tree, node, nodeT, cutoff=None)              # Projection chain from ROOT to each Terminal node is encoded as list of nodes in the chain
                for item in pathx:
                    size=len(item)-1
            size_chains.append(size)
        proj_degree=max(size_chains)                                    # Projection degree = No. of nodes in the longest projection chain from ROOT to a terminal node
        return proj_degree

    def projD_hist(self):
        projd_histogram={}
        for nodex in self.tree.nodes:
            projD=self.projection_degree(nodex)
            if projD in projd_histogram.keys():
                projd_histogram[projD]=projd_histogram[projD]+1
            else:
                projd_histogram[projD]=1
        return projd_histogram

    def arity(self):                                                    # Computes arity of the tree using out-degree of nodes
        tree_arity=self.tree.out_degree(list(self.tree.nodes))          # returns a dictionary containing nodenames as keys and its out-degree (or arity) as its values
        max_arity=max([x[1] for x in tree_arity])                       # Maximum out-degree = maximum arity in the tree
        avg_arity=sum([x[1] for x in tree_arity])/len([x[1] for x in tree_arity])
        histogram={}
        for arityx in [x[1] for x in tree_arity]:
            if arityx in histogram.keys():
                histogram[arityx]=histogram[arityx]+1                   # Creates arity histogram i.e. frequency of each arity
            else:
                histogram[arityx]=1
        arity_histogram=histogram
        return [max_arity, avg_arity, tree_arity, arity_histogram]

    def endpoint_crossing(self,edge):
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        endpoint={}

        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:       # nodes intervening the edge span which are not dominated by the any node in the edge span
                if not nodeI in nx.descendants(self.tree, edge[0]):
                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                        endpoint[self.tree.nodes[nodeI]['head']]=1        # creates a dictionary of all nodes having their outside their span. This dictionary has keys as 'heads' of the intervening nodes

        endpoint_cross=len(endpoint)                                   # If the intervening nodes belongs to more than head outside the edge span, then 1-endpoint crossing constraint is voilated
        return endpoint_cross                                          # returns the no. of heads outside the edge span which dominates the intervening nodes

    def compute_all(self):
        Arity = self.arity()
        Projection_degree = {node: self.projection_degree(node) for node in self.tree.nodes}
        Gap_degree = {node: self.gap_degree(node) for node in self.tree.nodes}
        direction = {}
        dep_distance = {}
        projectivity = {}
        Edge_degree = {}
        endpoint_cross = {}
        for edgex in self.tree.edges:
            direction[edgex] = self.dependency_direction(edgex)
            dep_distance[edgex] = self.dependency_distance(edgex)
            projectivity[edgex] = self.is_projective(edgex)
            Edge_degree[edgex] = self.edge_degree(edgex)
            endpoint_cross[edgex] = self.endpoint_crossing(edgex)

        return [Arity, Projection_degree, Gap_degree, direction, dep_distance, projectivity, Edge_degree, endpoint_cross]

    def all_dependent_constraint(self,edge):
        all_dep_deg=0
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)

        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        if not self.is_projective(edge):
            for nodey in edge_span:
                if not (edge[0] in nx.ancestors(self.tree, nodey)):                                                                 # if a node 'x' occurring the edge span is not dominated by the head of the edge
                    if not self.tree.nodes[nodey]['deprel']=='punct':
                        int_node=nodey
                        if not int_node==0:
                            for edgex in self.tree.edges:
                                if edgex[1]==int_node:
                                    int_head=edgex[0]
                            dep_int=0
                            for nodeI in edge_span:
                                if self.tree.nodes[nodeI]['head']==int_head:
                                    dep_int=dep_int+1
                                else:
                                    dep_int=dep_int
                            all_dep=0
                            for nodeJ in nx.descendants(self.tree, self.root):
                                if self.tree.nodes[nodeJ]['head']==int_head:
                                    all_dep=all_dep+1
                                else:
                                    all_dep=all_dep
                            all_dep_deg=all_dep-dep_int
                        else:
                            all_dep_deg=0
        else:
            all_dep_deg=100

        return all_dep_deg


    def hdd(self,edge):
        HDD=0
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)

        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        if not self.is_projective(edge):
            for nodeI in edge_span:
                if not self.tree.nodes[nodeI]['head'] in edge_span:       # nodes intervening the edge span which are not dominated by the any node in the edge span
                    if not nodeI in nx.descendants(self.tree, edge[0]):
                        if not self.tree.nodes[nodeI]['deprel']=='punct':
                            int_node=nodeI
                            if not int_node==0:
                                for edgex in self.tree.edges:
                                    if edgex[1]==int_node:
                                        int_head=edgex[0]
                                if nx.has_path(self.tree, int_head, edge[0]):
                                    pathx=nx.all_simple_paths(self.tree, int_head, edge[0], cutoff=None)
                                    for item in pathx:
                                        HDD=len(item)-1
                                elif nx.has_path(self.tree, edge[0], int_head):
                                    pathx=nx.all_simple_paths(self.tree, edge[0], int_head, cutoff=None)
                                    for item in pathx:
                                        HDD=len(item)-1
                                else:
                                    HDD=1
                            else:
                                HDD=2

        return HDD


In [ ]:
# Function to extract sentences from CoNLL-U format data
import io
def extract_sentences(data):
    sentences = []
    sentence = []
    for line in data.split('\n'):
        line = line.strip()
        if line.startswith('#'):  # Skip comments
            continue
        if line == '':  # End of sentence
            if sentence:
                sentences.append(sentence)
                sentence = []
        else:
            parts = line.split('\t')
            sentence.append(parts)
    return sentences

# Function to create tree representation of each sentence
def create_trees(sentences):
    trees = []
    for sentence in sentences:
        tree = nx.DiGraph()
        print("Processing sentence:", sentence)
        for token in sentence:
            print("Token:", token)
            try:
                head = int(token[6])
                dependent = int(token[0])
                print("Head:", head)
                print("Dependent:", dependent)
                if head > 0:  # Skip punctuation tokens
                    tree.add_node(dependent)
                    tree.add_node(head)
                    tree.add_edge(head, dependent)
                    print("Added edge:", (head, dependent))
            except (ValueError, IndexError) as e:
                print("Skipping token:", token)
                print("Error:", e)
        print("Nodes:", tree.nodes())
        print("Edges:", tree.edges())
        trees.append(tree)
    return trees

# Function to compute measures for each sentence
def compute_measures(trees):
    for tree in trees:
        print("Nodes:", tree.nodes())
        print("Edges:", tree.edges())
        # Add your code to compute measures for each tree

# Assuming you have loaded the sentences from the CONLLU file into the 'sentences' variable
trees = create_trees(sentences)
compute_measures(trees)
if upload.value:
    # Get the uploaded CONLLU file content
    uploaded_file = upload.value[next(iter(upload.value))]['content']

    # Decode the content from bytes to string
    uploaded_text = io.StringIO(uploaded_file.decode('utf-8')).read()

    # Extract sentences from the parsed data
    sentences = extract_sentences(uploaded_text)

    # Create tree representation for each sentence
    trees = create_trees(sentences)

    # Compute measures for each sentence
    results = compute_measures(trees)

    # Now results contains the measures for each sentence
    # You can iterate over it to access the measures for each sentence
else:
    print("No file uploaded. Please upload a CoNLL-U format file.")


NameError: name 'sentences' is not defined

In [ ]:
def extract_nodes_and_edges(conllu_content):
    sentences = conllu_content.strip().split('\n\n')
    all_nodes = []
    all_edges = []

    for sentence in sentences:
        lines = sentence.strip().split('\n')
        nodes = []
        edges = []

        for line in lines:
            if line.startswith('#'):
                continue
            parts = line.split('\t')
            node_id = parts[0]
            word = parts[1]
            pos = parts[3]
            head = parts[5]
            nodes.append((node_id, word))
            edges.append((head, node_id))

        all_nodes.append(nodes)
        all_edges.append(edges)

    return all_nodes, all_edges

# Get the uploaded CONLLU file content
uploaded_file_content = upload.value[next(iter(upload.value))]['content'].decode('utf-8')

# Extract nodes and edges
all_nodes, all_edges = extract_nodes_and_edges(uploaded_file_content)

# Print node and edge information for each sentence
for i, (nodes, edges) in enumerate(zip(all_nodes, all_edges)):
    print(f"Sentence {i + 1}:")
    print("Nodes:")
    for node in nodes:
        print(node)
    print("Edges:")
    for edge in edges:
        print(edge)
    print()

Streaming output truncated to the last 5000 lines.
('42', '40')
('42', '41')
('44', '42')
('44', '43')
('60', '44')
('44', '45')
('45', '46')
('44', '47')
('60', '48')
('50', '49')
('5', '50')
('60', '51')
('53', '52')
('51', '53')
('53', '54')
('60', '55')
('58', '56')
('58', '57')
('60', '58')
('60', '59')
('0', '60')
('60', '61')
('2', '1')
('74', '2')
('4', '3')
('2', '4')
('6', '5')
('2', '6')
('8', '7')
('2', '8')
('10', '9')
('2', '10')
('12', '11')
('2', '12')
('15', '13')
('15', '14')
('2', '15')
('17', '16')
('20', '17')
('19', '18')
('20', '19')
('2', '20')
('23', '21')
('23', '22')
('25', '23')
('25', '24')
('26', '25')
('29', '26')
('29', '27')
('29', '28')
('35', '29')
('29', '30')
('33', '31')
('33', '32')
('35', '33')
('35', '34')
('43', '35')
('38', '36')
('38', '37')
('39', '38')
('40', '39')
('43', '40')
('43', '41')
('43', '42')
('48', '43')
('43', '44')
('48', '45')
('45', '46')
('48', '47')
('49', '48')
('2', '49')
('48', '50')
('49', '51')
('51', '52')
('56', '53

In [ ]:
uploaded_file_content = upload.value[next(iter(upload.value))]['content'].decode('utf-8')
sentences = uploaded_file_content.strip().split('\n\n')
for sentence in sentences:
  print(sentence+"\n\n\n\n")

1	Holly	Holly	PROPN	NNP	3	compound	0	5	2
2	Bluff	Bluff	PROPN	NNP	1	flat	7	12	1
3	Church	Church	PROPN	NNP	4	nsubj	14	20	1
4	wishes	wish	VERB	VBZ	0	root	22	28	4
5	to	to	PART	TO	6	mark	30	32	1
6	know	know	VERB	VB	4	xcomp	34	38	2
7	if	if	SCONJ	IN	10	mark	40	42	3
8	any	any	DET	DT	9	det	44	47	1
9	church	church	NOUN	NN	10	nsubj	48	54	1
10	has	have	VERB	VBZ	6	ccomp	56	59	4
11	church	church	NOUN	NN	12	compound	61	67	1
12	furniture	furniture	NOUN	NN	10	obj	69	78	2
13	it	it	PRON	PRP	15	nsubj	80	82	2
14	is	be	AUX	VBZ	15	aux	84	86	1
15	taking	take	VERB	VBG	12	acl:relcl	88	94	3
16	out	out	ADP	RP	15	compound:prt	96	99	1
17	to	to	PART	TO	18	mark	101	103	1
18	give	give	VERB	VB	15	advcl	104	108	3
19	place	place	NOUN	NN	18	obj	110	115	1
20	to	to	ADP	IN	22	case	117	119	2
21	new	new	ADJ	JJ	22	amod	121	124	1
22	furniture	furniture	NOUN	NN	18	obl	126	135	4
23	.	.	PUNCT	.	4	punct	135	136	19




1	If	if	SCONJ	IN	3	mark	138	140	2
2	you	you	PRON	PRP	3	nsubj	142	145	1
3	have	have	AUX	VBP	4	aux	147	151	1
4	used	us

In [ ]:
def extract_nodes_and_edges(conllu_content):
    sentences = conllu_content.strip().split('\n\n')
    all_nodes = []
    all_edges = []
    sent_id = 0
    for sentence in sentences:
        sent_id += 1
        lines = sentence.strip().split('\n')
        nodes = []
        edges = []

        for line in lines:
            if line.startswith('#'):
                continue
            parts = line.split('\t')
            node_id = parts[0]
            word = parts[1]
            pos = parts[3]
            head = parts[5]
            nodes.append((node_id, word, pos, sent_id))
            if head != '0':  # Only add edges if head is not '0'
                edges.append((head, node_id))

        all_nodes.append(nodes)
        all_edges.append(edges)

    return all_nodes, all_edges

def generate_latex_dependency_trees(all_nodes, all_edges):
    latex_code = ""

    for nodes, edges in zip(all_nodes, all_edges):
        latex_code += "\\begin{dependency}[theme = simple]\n"
        latex_code += "\\begin{deptext}[column sep=1em]\n"

        # Add words in the dependency tree
        words = " \& ".join([word for _, word, _, _ in nodes])
        latex_code += f"{words} \\\\\n\\end{{deptext}}\n"

        # Add dependency edges
        for head, dependent in edges:
            latex_code += f"\\depedge{{{head}}}{{{dependent}}}{{}}\n"

        latex_code += "\\end{dependency}\n\n"

    return latex_code

# Assuming you have uploaded the CoNLL-U file content
uploaded_file_content = upload.value[next(iter(upload.value))]['content'].decode('utf-8')

# Extract nodes and edges
all_nodes, all_edges = extract_nodes_and_edges(uploaded_file_content)

# Generate LaTeX code for dependency trees
latex_code = generate_latex_dependency_trees(all_nodes, all_edges)

# Print the LaTeX code
print(latex_code)


\begin{dependency}[theme = simple]
\begin{deptext}[column sep=1em]
Holly \& Bluff \& Church \& wishes \& to \& know \& if \& any \& church \& has \& church \& furniture \& it \& is \& taking \& out \& to \& give \& place \& to \& new \& furniture \& . \\
\end{deptext}
\depedge{3}{1}{}
\depedge{1}{2}{}
\depedge{4}{3}{}
\depedge{6}{5}{}
\depedge{4}{6}{}
\depedge{10}{7}{}
\depedge{9}{8}{}
\depedge{10}{9}{}
\depedge{6}{10}{}
\depedge{12}{11}{}
\depedge{10}{12}{}
\depedge{15}{13}{}
\depedge{15}{14}{}
\depedge{12}{15}{}
\depedge{15}{16}{}
\depedge{18}{17}{}
\depedge{15}{18}{}
\depedge{18}{19}{}
\depedge{22}{20}{}
\depedge{22}{21}{}
\depedge{18}{22}{}
\depedge{4}{23}{}
\end{dependency}

\begin{dependency}[theme = simple]
\begin{deptext}[column sep=1em]
If \& you \& have \& used \& church \& furniture \& to \& dispose \& of \& write \& F. \& B. \& Hart \& , \& Holly \& Bluff \& . \\
\end{deptext}
\depedge{3}{1}{}
\depedge{3}{2}{}
\depedge{4}{3}{}
\depedge{6}{5}{}
\depedge{4}{6}{}
\depedge{8}{7

In [ ]:
def extract_nodes_and_edges(conllu_content):
    sentences = conllu_content.strip().split('\n\n')
    all_nodes = []
    all_edges = []
    sent_id = 0
    for sentence in sentences:
        sent_id += 1
        lines = sentence.strip().split('\n')
        nodes = []
        edges = []

        for line in lines:
            if line.startswith('#'):
                continue
            parts = line.split('\t')
            node_id = parts[0]
            word = parts[1]
            pos = parts[3]
            head = parts[5]
            nodes.append((node_id, word, pos, sent_id))
            edges.append((head, node_id))

        all_nodes.append(nodes)
        all_edges.append(edges)

    return all_nodes, all_edges

# Get the uploaded CONLLU file content
uploaded_file_content = upload.value[next(iter(upload.value))]['content'].decode('utf-8')

# Extract nodes and edges
all_nodes, all_edges = extract_nodes_and_edges(uploaded_file_content)

# Print node and edge information for each sentence
for i, (nodes, edges) in enumerate(zip(all_nodes, all_edges)):
    print(f"Sentence {i + 1}:")
    print("Nodes:")
    for node in nodes:
        print(node)
    print("Edges:")
    for edge in edges:
        print(edge)
    print()

def generate_latex_dependency_trees(all_nodes, all_edges):
    latex_code = ""
    sent_ids=[]
    for nodes in all_nodes:
        sent_ids.append(nodes[sent_id])
    for i in sent_ids:
        latex_code += f"\\begin{{dependency}}[theme = simple]\n"
        latex_code += f"\\begin{{deptext}}[column sep=1em]\n"
        for i, (nodes, edges) in enumerate(zip(all_nodes, all_edges)):
            if nodes[sent_id]==1:
                for node_id, word, _ in nodes:  # Change pos to _
                    latex_code += f"{word} \& "
            latex_code += "\\\\\n\\end{deptext}\n"
            if nodes[sent_id]==1:
                for head, dependent in edges:
                    latex_code += f"\\depedge{{{head}}}{{{dependent}}}{{}}\n"
        latex_code += "\\end{dependency}\n\n"
    return latex_code

# Assuming you have extracted nodes and edges from the CoNLL-U format file
# all_nodes and all_edges should be lists containing nodes and edges for each sentence respectively

# Generate LaTeX code for dependency trees
latex_code = generate_latex_dependency_trees(all_nodes, all_edges)

# Print the LaTeX code
print(latex_code)

Sentence 1:
Nodes:
('1', 'Holly', 'PROPN', 1)
('2', 'Bluff', 'PROPN', 1)
('3', 'Church', 'PROPN', 1)
('4', 'wishes', 'VERB', 1)
('5', 'to', 'PART', 1)
('6', 'know', 'VERB', 1)
('7', 'if', 'SCONJ', 1)
('8', 'any', 'DET', 1)
('9', 'church', 'NOUN', 1)
('10', 'has', 'VERB', 1)
('11', 'church', 'NOUN', 1)
('12', 'furniture', 'NOUN', 1)
('13', 'it', 'PRON', 1)
('14', 'is', 'AUX', 1)
('15', 'taking', 'VERB', 1)
('16', 'out', 'ADP', 1)
('17', 'to', 'PART', 1)
('18', 'give', 'VERB', 1)
('19', 'place', 'NOUN', 1)
('20', 'to', 'ADP', 1)
('21', 'new', 'ADJ', 1)
('22', 'furniture', 'NOUN', 1)
('23', '.', 'PUNCT', 1)
Edges:
('3', '1')
('1', '2')
('4', '3')
('0', '4')
('6', '5')
('4', '6')
('10', '7')
('9', '8')
('10', '9')
('6', '10')
('12', '11')
('10', '12')
('15', '13')
('15', '14')
('12', '15')
('15', '16')
('18', '17')
('15', '18')
('18', '19')
('22', '20')
('22', '21')
('18', '22')
('4', '23')

Sentence 2:
Nodes:
('1', 'If', 'SCONJ', 2)
('2', 'you', 'PRON', 2)
('3', 'have', 'AUX', 2)
('4', 'u

NameError: name 'sent_id' is not defined

In [ ]:
def extract_nodes_and_edges(conllu_content):
    sentences = conllu_content.strip().split('\n\n')
    all_nodes = []
    all_edges = []
    sent_id = 0
    for sentence in sentences:
        sent_id += 1
        lines = sentence.strip().split('\n')
        nodes = []
        edges = []

        for line in lines:
            if line.startswith('#'):
                continue
            parts = line.split('\t')
            node_id = parts[0]
            word = parts[1]
            pos = parts[3]
            head = parts[6]  # corrected from 5 to 6
            nodes.append((node_id, word, pos, sent_id))
            edges.append((head, node_id))

        all_nodes.append(nodes)
        all_edges.append(edges)

    return all_nodes, all_edges

# Get the uploaded CONLLU file content
uploaded_file_content = upload.value[next(iter(upload.value))]['content'].decode('utf-8')

# Extract nodes and edges
all_nodes, all_edges = extract_nodes_and_edges(uploaded_file_content)

# Print node and edge information for each sentence
for i, (nodes, edges) in enumerate(zip(all_nodes, all_edges)):
    print(f"Sentence {i + 1}:")
    print("Nodes:")
    for node in nodes:
        print(node)
    print("Edges:")
    for edge in edges:
        print(edge)
    print()

def generate_latex_dependency_trees(all_nodes, all_edges):
    latex_code = ""

    for i, (nodes, edges) in enumerate(zip(all_nodes, all_edges)):
        latex_code += f"\\begin{{dependency}}[theme = simple]\n"
        latex_code += f"\\begin{{deptext}}[column sep=1em]\n"

        # Collect words for the current sentence
        words = " \\& ".join([word for node_id, word, pos, sent_id in nodes])
        latex_code += f"{words} \\\\\n\\end{{deptext}}\n"

        # Collect edges for the current sentence
        for head, dependent in edges:
            latex_code += f"\\depedge{{{head}}}{{{dependent}}}{{}}\n"

        latex_code += "\\end{dependency}\n\n"

    return latex_code

# Assuming you have extracted nodes and edges from the CoNLL-U format file
# all_nodes and all_edges should be lists containing nodes and edges for each sentence respectively

# Generate LaTeX code for dependency trees
latex_code = generate_latex_dependency_trees(all_nodes, all_edges)

# Print the LaTeX code
print(latex_code)


NameError: name 'upload' is not defined

In [ ]:
# Create a sample tree (modify based on actual DOC content)
tree = nx.DiGraph()
tree.add_node(0, form='ROOT', POS='ROOT')
tree.add_node(1, form='this', POS='PRN')
tree.add_node(2, form='is', POS='VBZ')
tree.add_node(3, form='a', POS='DT')
tree.add_node(4, form='test', POS='NN')
tree.add_edges_from([(0, 1), (1, 2), (2, 3), (3, 4)])

# Initialize the Compute_measures class with the tree
compute_measures = Compute_measures(tree)

# Example of using the class methods
print(compute_measures.compute_all())

[[1, 0.8, OutDegreeView({0: 1, 1: 1, 2: 1, 3: 1, 4: 0}), {1: 4, 0: 1}], {0: 4, 1: 3, 2: 2, 3: 1, 4: 0}, {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}, {(0, 1): 'LR', (1, 2): 'LR', (2, 3): 'LR', (3, 4): 'LR'}, {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}, {(0, 1): True, (1, 2): True, (2, 3): True, (3, 4): True}, {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}, {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}]


In [ ]:
def output_to_latex(output):
    print("Inside output_to_latex function")
    latex_output = []
    # Rest of the function code...


In [ ]:
def output_to_latex(output):
    latex_output = []
    for result in output:
        if isinstance(result, dict):
            print("Dict part found")
            for item in result.items():
                print(item)
        elif isinstance(result, list) and len(result) == 4:
            # Handle the list part (usually the node properties and structure)
            _, _, outdegree_view, node_info = result
            nodes = list(outdegree_view.keys())
            node_text = ' & '.join(str(node) for node in nodes) + ' \\\\'
            latex_output.append(f"\\begin{{deptext}}[column sep=1em]\n{node_text}\n\\end{{deptext}}")
        else:
            # Otherwise, it might be node degree information, handle it accordingly
            for node, value in result.items():
                # Handle the node degree part
                if isinstance(value, int):
                    # Ignore nodes with no edges or with zero degree
                    if value > 0:
                        latex_output.append(f"\\deproot{{{node + 1}}}{{}}")
                else:
                  print(f"Unexpected type: {type(result)}")

    return latex_output



def write_latex_document(output, filename):
    latex_header = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{geometry}
\geometry{a3paper}
\usepackage{tikz}
\usepackage{tikz-dependency}

\begin{document}

\centering
\large
"""
    latex_footer = r"""
\end{document}
"""
    latex_code = output_to_latex(output)
    latex_content = '\n'.join(latex_code)

    with open(filename, 'w') as f:
        f.write(latex_header)
        f.write(latex_content)
        f.write(latex_footer)

# Example output from compute_all function
compute_all_output = [
    [1, 0.8, {0: 1, 1: 1, 2: 1, 3: 1, 4: 0}, {1: 4, 0: 1}],
    {0: 4, 1: 3, 2: 2, 3: 1, 4: 0},
    {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
    {(0, 1): 'LR', (1, 2): 'LR', (2, 3): 'LR', (3, 4): 'LR'},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0},
    {(0, 1): True, (1, 2): True, (2, 3): True, (3, 4): True},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}
]

# Write the LaTeX document
write_latex_document(compute_all_output, 'dependency_trees.tex')


Dict part found
(0, 4)
(1, 3)
(2, 2)
(3, 1)
(4, 0)
Dict part found
(0, 0)
(1, 0)
(2, 0)
(3, 0)
(4, 0)
Dict part found
((0, 1), 'LR')
((1, 2), 'LR')
((2, 3), 'LR')
((3, 4), 'LR')
Dict part found
((0, 1), 0)
((1, 2), 0)
((2, 3), 0)
((3, 4), 0)
Dict part found
((0, 1), True)
((1, 2), True)
((2, 3), True)
((3, 4), True)
Dict part found
((0, 1), 0)
((1, 2), 0)
((2, 3), 0)
((3, 4), 0)
Dict part found
((0, 1), 0)
((1, 2), 0)
((2, 3), 0)
((3, 4), 0)


In [ ]:
def generate_latex_dependency_tree(output):
    latex_code = ""

    for result in output:
        # Start a new dependency tree
        latex_code += "\\begin{dependency}[theme=simple]\n"
        latex_code += "\\begin{deptext}[column sep=1em]\n"

        if isinstance(result, list):
            # Add the tokens from a list
            for token in result:
                latex_code += f"{token} "
                latex_code += "\& "
            latex_code = latex_code[:-2]  # Remove the last '\&' and add a new line
            latex_code += "\\\\\n"

        elif isinstance(result, dict):
            # Add the tokens from a dictionary
            for token_pos, token in result.items():
                latex_code += f"{token} "
                if token_pos < max(result.keys()):
                    latex_code += "\\& "
                else:
                    latex_code += "\\\\\n"

        latex_code += "\\end{deptext}\n"

        # Add dependency edges
        if isinstance(result, dict):
            for edge, value in result.items():
                if isinstance(edge, tuple):
                    node1, node2 = edge
                    if value == 'LR':
                        latex_code += f"\\depedge{{{node1}}}{{{node2}}}{{}}"
                    elif isinstance(value, bool) and value:
                        latex_code += f"\\depedge{{{node1}}}{{{node2}}}{{}}"
                    else:
                        latex_code += f"\\depedge{{{node1}}}{{{node2}}}{{{value}}}"
                    latex_code += "\n"

        latex_code += "\\end{dependency}\n\n"

    return latex_code

# Example output from compute_all function
compute_all_output = [
    {(0, 1): 'LR', (1, 2): 'LR', (2, 3): 'LR', (3, 4): 'LR'},
    {0: 4, 1: 3, 2: 2, 3: 1, 4: 0},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0},
    {(0, 1): True, (1, 2): True, (2, 3): True, (3, 4): True},
]

# Generate LaTeX code for dependency trees
latex_code = generate_latex_dependency_tree(compute_all_output)


print(latex_code)

# Write LaTeX code to a file
with open("dependency_trees.tex", "w") as f:
    f.write(latex_code)




\begin{dependency}[theme=simple]
\begin{deptext}[column sep=1em]
LR \& LR \& LR \& LR \\
\end{deptext}
\depedge{0}{1}{}
\depedge{1}{2}{}
\depedge{2}{3}{}
\depedge{3}{4}{}
\end{dependency}

\begin{dependency}[theme=simple]
\begin{deptext}[column sep=1em]
4 \& 3 \& 2 \& 1 \& 0 \\
\end{deptext}
\end{dependency}

\begin{dependency}[theme=simple]
\begin{deptext}[column sep=1em]
0 \& 0 \& 0 \& 0 \\
\end{deptext}
\depedge{0}{1}{0}
\depedge{1}{2}{0}
\depedge{2}{3}{0}
\depedge{3}{4}{0}
\end{dependency}

\begin{dependency}[theme=simple]
\begin{deptext}[column sep=1em]
True \& True \& True \& True \\
\end{deptext}
\depedge{0}{1}{}
\depedge{1}{2}{}
\depedge{2}{3}{}
\depedge{3}{4}{}
\end{dependency}




In [ ]:
print(compute_all_output)


[[1, 0.8, {0: 1, 1: 1, 2: 1, 3: 1, 4: 0}, {1: 4, 0: 1}], {0: 4, 1: 3, 2: 2, 3: 1, 4: 0}, {0: 0, 1: 0, 2: 0, 3: 0, 4: 0}, {(0, 1): 'LR', (1, 2): 'LR', (2, 3): 'LR', (3, 4): 'LR'}, {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}, {(0, 1): True, (1, 2): True, (2, 3): True, (3, 4): True}, {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}, {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}]


In [ ]:
def output_to_latex(output):
    latex_output = []
    for result in output:
        if isinstance(result, dict):
            for (node1, node2), value in result.items():
                if isinstance(value, str):
                    if value == 'LR':
                        latex_output.append(f"\\depedge{{{node1 + 1}}}{{{node2 + 1}}}{{}}")
                    else:
                        latex_output.append(f"\\depedge{{{node1 + 1}}}{{{node2 + 1}}}{{{value}}}")
                elif isinstance(value, bool) and value:
                    latex_output.append(f"\\depedge{{{node1 + 1}}}{{{node2 + 1}}}{{}}")
                elif isinstance(value, int) and value == 0:
                    # Handle optional case where the weight is 0
                    latex_output.append(f"\\depedge[<optional_style>]{{{node1 + 1}}}{{{node2 + 1}}}{{{value}}}")
        elif isinstance(result, list) and len(result) == 4:
            _, _, outdegree_view, node_info = result
            nodes = list(outdegree_view.keys())
            node_text = ' & '.join(str(node) for node in nodes) + ' \\\\'
            latex_output.append(f"\\begin{{dependency}}[theme=simple]")
            latex_output.append(f"\\begin{{deptext}}[column sep=1em]\n{node_text}\n\\end{{deptext}}")
        else:
            for node, value in result.items():
                if isinstance(value, int) and value > 0:
                    latex_output.append(f"\\deproot{{{node + 1}}}{{}}")

    return latex_output

def write_latex_document(output, filename):
    latex_header = r"""
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{geometry}
\geometry{a3paper}
\usepackage{tikz}
\usepackage{tikz-dependency}

\begin{document}

\centering
\large
"""
    latex_footer = r"""
\end{document}
"""
    latex_code = output_to_latex(output)
    latex_content = '\n'.join(latex_code)

    with open(filename, 'w') as f:
        f.write(latex_header)
        f.write(latex_content)
        f.write(latex_footer)

compute_all_output = [
    [1, 0.8, {0: 1, 1: 1, 2: 1, 3: 1, 4: 0}, {1: 4, 0: 1}],
    {0: 4, 1: 3, 2: 2, 3: 1, 4: 0},
    {0: 0, 1: 0, 2: 0, 3: 0, 4: 0},
    {(0, 1): 'LR', (1, 2): 'LR', (2, 3): 'LR', (3, 4): 'LR'},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0},
    {(0, 1): True, (1, 2): True, (2, 3): True, (3, 4): True},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0},
    {(0, 1): 0, (1, 2): 0, (2, 3): 0, (3, 4): 0}
]

# Write the LaTeX document
write_latex_document(compute_all_output, 'dependency_trees.tex')


TypeError: cannot unpack non-iterable int object

In [ ]:
class Compute_measures(object):
    def __init__(self, tree):
        self.tree=tree                          # tree encodes the nodes and edges content in dictionary format. It uses directed graph (DiGraph) feature of networkX package. For example, nodes are encoded like this - tree.nodes={1:{form:'this',POS:'PRN'},2:{...}}
        self.root=0                             # ROOT is an abstract node in the tree and is encoded as empty and with name=0

    def dependency_direction(self, edge):       # Computes the direction of an edge (i.e., dependency) according to relative position of dependent and head
        direction=''
        if edge[0]>edge[1]:                     # edge is a list data type in the format [head,dependent]
            direction='RL'
        else:
            direction='LR'

        return direction                        # return direction as 'LR' (Left-to-Right) or RL ('Right-to-Left')


In [ ]:
def dependency_distance(self, edge):        # Computes the dependency length i.e., no. of nodes between head and its dependent
        dd=0
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:                             # all the nodes that lies linearly between dependent and head
                    dd+=1
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    dd+=1
        return dd                              # returns the dependency distance of the edge



In [ ]:
def dependency_depth(self, edge):        # Computes the dependency length i.e., no. of nodes between head and its dependent
        hd=1
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:                             # all the nodes that lies linearly between dependent and head
                    if nx.descendants(self.tree, nodex):
                        hd+=1
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    if nx.descendants(self.tree, nodex):
                        hd+=1
        return hd                              # returns the dependency distance of the edge


In [ ]:
 def is_projective(self, edge):             # Checks if an edge is projective or not and returns a boolean value.
        projective=True
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        flag=0
        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:
                if not nodeI in nx.descendants(self.tree, edge[0]):
                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                        flag += 1
        if not flag==0:
            projective=False
        return projective


In [ ]:
def edge_degree(self, edge):                                 # Computes the number of edges causing non-projectivity
        eD=0
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:         # if the head of any intervening node exists outside the span of the edge
                if not nodeI in nx.descendants(self.tree, edge[0]):
                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                        eD += 1
        return eD


In [ ]:
def gap_degree(self, node):                     # Computes the gaps in the projection chain containing maximum number of gaps
        chains_gapD=[]
        terminals=[]
        for nodex in self.tree.nodes:
            if self.tree.out_degree(nodex)==0:
                terminals.append(nodex)

        for nodeT in terminals:
            gapD=0
            if nx.has_path(self.tree, node, nodeT):
                pathx=nx.all_simple_paths(self.tree, node, nodeT, cutoff=None)     # Projection chain from ROOT to each Terminal node is encoded as list of nodes in the chain
                for item in pathx:
                    pathy=item
                    for nodeP in item:
                        if not nodeP==self.root:
                            if not self.tree.nodes[nodeP]['head']==self.root:
                                if not self.is_projective([self.tree.nodes[nodeP]['head'],nodeP]):   # If any edge in a projection chain is non-projective
                                    gapD=gapD+1                                                      # No. of non-projective edges in a projection chain = No. of gap degree
            chains_gapD.append(gapD)

        gap_deg=max(chains_gapD)                                                             # Gap degree of a node is the maximum gap degree in its dependents
        return gap_deg

In [ ]:
  def gapnodes(self, pathx):                                          # Returns all the nodes which causes GAP in the projection of a node
        nodeM=[]
        for nodex in pathx:
            if not nodex==self.root:
                if not self.tree.nodes[nodex]['head']==self.root:
                    if not self.is_projective([self.tree.nodes[nodex]['head'],nodex]):
                        cross_dep=nodex
                        cross_head=self.tree.nodes[cross_dep]['head']
                        edge_span=[]
                        if cross_head>cross_dep:
                            for nodev in nx.descendants(self.tree, self.root):
                                if cross_dep<nodev<cross_head:
                                    edge_span.append(nodev)
                        else:
                            for nodev in nx.descendants(self.tree, self.root):
                                if cross_head<nodev<cross_dep:
                                    edge_span.append(nodev)
                        for nodeI in edge_span:
                            if not self.tree.nodes[nodeI]['head'] in edge_span:         # if the head of any intervening node exists outside the span of the edge
                                if not nodeI in nx.descendants(self.tree, cross_head):
                                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                                        nodeM.append(nodeI)
        return nodeM

In [ ]:
 def illnestedness(self,node,gapD):
        k_illnest=0
        if gapD==0:
            k_illnest=0
        else:
            illnest = []
            all_gapped_chains=[]
            for nodex in nx.descendants(self.tree, self.root):
                if not nodex==self.root:
                    if not self.tree.nodes[nodex]['head']==self.root:
                        if not self.is_projective([self.tree.nodes[nodex]['head'],nodex]):
                            if nx.has_path(self.tree, node, nodex):
                                pathx=nx.all_simple_paths(self.tree, node, nodex, cutoff=None)
                                for item in pathx:
                                    all_gapped_chains.append(item)

                        chains_with_gaps=[]
            for chainx in all_gapped_chains:
                flag=0
                for chainy in all_gapped_chains:
                    if set(chainx) < set(chainy):
                        flag=flag+1
                if flag==0:
                    chains_with_gaps.append(chainx)

            for pathz in chains_with_gaps:
                #print(pathz)
                num_interL=0                                                        # Variable for number of chains that can interleave with a single chain in question
                nodeM=self.gapnodes(pathz)
                for pathy in chains_with_gaps:
                    if not pathy==pathz:
                        flag=0
                        for nodeC in nodeM:
                            if nodeC in pathy:
                                flag=flag+1
                        if not flag==0:
                            nodeMM=self.gapnodes(pathy)
                            flagg=0
                            for nodeCC in nodeMM:
                                if nodeCC in pathz:
                                    flagg=flagg+1
                            if not flagg==0:
                                num_interL = num_interL + 1
                illnest.append(num_interL)
            k_illnest = max(illnest)
        return k_illnest



In [ ]:
    def gapD_hist(self):
        gapd_histogram={}
        for nodex in self.tree.nodes:
            gapD=self.gap_degree(nodex)
            if gapD in gapd_histogram.keys():
                gapd_histogram[gapD]=gapd_histogram[gapD]+1
            else:
                gapd_histogram[gapD]=1
        return gapd_histogram

    def projection_degree(self, node):
        size_chains=[]
        terminals=[]
        for nodex in self.tree.nodes:
            if self.tree.out_degree(nodex)==0:
                terminals.append(nodex)


        for nodeT in terminals:
            size=0
            if nx.has_path(self.tree, node, nodeT):
                pathx=nx.all_simple_paths(self.tree, node, nodeT, cutoff=None)              # Projection chain from ROOT to each Terminal node is encoded as list of nodes in the chain
                for item in pathx:
                    size=len(item)-1
            size_chains.append(size)
        proj_degree=max(size_chains)                                    # Projection degree = No. of nodes in the longest projection chain from ROOT to a terminal node
        return proj_degree

    def projD_hist(self):
        projd_histogram={}
        for nodex in self.tree.nodes:
            projD=self.projection_degree(nodex)
            if projD in projd_histogram.keys():
                projd_histogram[projD]=projd_histogram[projD]+1
            else:
                projd_histogram[projD]=1
        return projd_histogram

In [ ]:
    def arity(self):                                                    # Computes arity of the tree using out-degree of nodes
        tree_arity=self.tree.out_degree(list(self.tree.nodes))          # returns a dictionary containing nodenames as keys and its out-degree (or arity) as its values
        max_arity=max([x[1] for x in tree_arity])                       # Maximum out-degree = maximum arity in the tree
        avg_arity=sum([x[1] for x in tree_arity])/len([x[1] for x in tree_arity])
        histogram={}
        for arityx in [x[1] for x in tree_arity]:
            if arityx in histogram.keys():
                histogram[arityx]=histogram[arityx]+1                   # Creates arity histogram i.e. frequency of each arity
            else:
                histogram[arityx]=1
        arity_histogram=histogram
        return [max_arity, avg_arity, tree_arity, arity_histogram]

    def endpoint_crossing(self,edge):
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)
        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        endpoint={}
        for nodeI in edge_span:
            if not self.tree.nodes[nodeI]['head'] in edge_span:       # nodes intervening the edge span which are not dominated by the any node in the edge span
                if not nodeI in nx.descendants(self.tree, edge[0]):
                    if not self.tree.nodes[nodeI]['deprel']=='punct':
                        endpoint[self.tree.nodes[nodeI]['head']]=1        # creates a dictionary of all nodes having their outside their span. This dictionary has keys as 'heads' of the intervening nodes

        endpoint_cross=len(endpoint)                                   # If the intervening nodes belongs to more than head outside the edge span, then 1-endpoint crossing constraint is voilated
        return endpoint_cross                                          # returns the no. of heads outside the edge span which dominates the intervening nodes


In [ ]:
def compute_all(self):
        Arity=self.arity()
        Projection_degree=self.projection_degree()
        Gap_degree=self.gap_degree()
        direction={}
        dep_distane={}
        projectivity={}
        Edge_degree={}
        endpoint_cross={}
        for edgex in self.tree.edges:
            direction[edgex]=self.dependency_direction(edgex)
            dep_distance[edgex]=self.dependency_distance(edgex)
            projectivity[edgex]=self.is_projective(edgex)
            Edge_degree[edgex]=self.edge_degree(edgex)
            endpoint_cross[edgex]=self.endpoint_crossing(edgex)

        return [Arity, Projection_degree, Gap_degree, direction, dep_distance, projectivity, Edge_degree, endpoint_cross]


In [ ]:
def all_dependent_constraint(self,edge):
        all_dep_deg=0
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)

        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        if not self.is_projective(edge):
            for nodey in edge_span:
                if not (edge[0] in nx.ancestors(self.tree, nodey)):                                                                 # if a node 'x' occurring the edge span is not dominated by the head of the edge
                    if not self.tree.nodes[nodey]['deprel']=='punct':
                        int_node=nodey
                        if not int_node==0:
                            for edgex in self.tree.edges:
                                if edgex[1]==int_node:
                                    int_head=edgex[0]
                            dep_int=0
                            for nodeI in edge_span:
                                if self.tree.nodes[nodeI]['head']==int_head:
                                    dep_int=dep_int+1
                                else:
                                    dep_int=dep_int
                            all_dep=0
                            for nodeJ in nx.descendants(self.tree, self.root):
                                if self.tree.nodes[nodeJ]['head']==int_head:
                                    all_dep=all_dep+1
                                else:
                                    all_dep=all_dep
                            all_dep_deg=all_dep-dep_int
                        else:
                            all_dep_deg=0
        else:
            all_dep_deg=100

        return all_dep_deg


In [ ]:
  def hdd(self,edge):
        HDD=0
        edge_span=[]
        if edge[0]>edge[1]:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[1]<nodex<edge[0]:
                    edge_span.append(nodex)

        else:
            for nodex in nx.descendants(self.tree, self.root):
                if edge[0]<nodex<edge[1]:
                    edge_span.append(nodex)

        if not self.is_projective(edge):
            for nodeI in edge_span:
                if not self.tree.nodes[nodeI]['head'] in edge_span:       # nodes intervening the edge span which are not dominated by the any node in the edge span
                    if not nodeI in nx.descendants(self.tree, edge[0]):
                        if not self.tree.nodes[nodeI]['deprel']=='punct':
                            int_node=nodeI
                            if not int_node==0:
                                for edgex in self.tree.edges:
                                    if edgex[1]==int_node:
                                        int_head=edgex[0]
                                if nx.has_path(self.tree, int_head, edge[0]):
                                    pathx=nx.all_simple_paths(self.tree, int_head, edge[0], cutoff=None)
                                    for item in pathx:
                                        HDD=len(item)-1
                                elif nx.has_path(self.tree, edge[0], int_head):
                                    pathx=nx.all_simple_paths(self.tree, edge[0], int_head, cutoff=None)
                                    for item in pathx:
                                        HDD=len(item)-1
                                else:
                                    HDD=1
                            else:
                                HDD=2

        return HDD

In [ ]:
from google.colab import files
import io

# Function to read the uploaded CoNLL format file
def read_uploaded_conll(uploaded_file):
    return io.StringIO(uploaded_file.decode('utf-8'))

# Function to process CoNLL data
def process_conll_data(conll_data):
    # Implement your processing logic here
    # For example:
    # results = measures_calculator.compute_all_from_conll(conll_data)
    # return results
    pass

# Upload CoNLL file
print("Upload a CoNLL format file:")
uploaded = files.upload()

# Process uploaded file
for file_name, content in uploaded.items():
    print("Processing file:", file_name)
    conll_data = read_uploaded_conll(content)
    results = process_conll_data(conll_data)
    print(results)


Upload a CoNLL format file:


Saving parsed_data.conll to parsed_data.conll
Processing file: parsed_data.conll
None
